In [ ]:
from qutip import sigmax
from qutip.control.grape import _overlap
from notebooks.qutip_wrapper import qutip_optimize_wrapper, qutip_simulate_wrapper
%matplotlib inline
# import matplotlib.pyplot as plt
import numpy as np
# import datetime
# from qutip_wrapper import qutip_optimize_wrapper, qutip_simulate_wrapper
from notebooks.qiskit_scripts import *
# from qutip import identity, sigmax, sigmaz, sigmay, hadamard_transform

from qiskit.tools.qi.qi import outer

In [ ]:
from qiskit import IBMQ, pulse, QuantumCircuit, ClassicalRegister
from qiskit.pulse import Play

IBMQ.load_account()

provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')
armonk_backend = backend

In [97]:
freq_est = 4974450442.586346
omegad0 = 31919806.545849085
dt = armonk_backend.configuration().dt
wq0 = 2 * np.pi * freq_est

n_ts = 1600
evo_time = dt * n_ts

default_delta = wq0/16
delta=default_delta
delta_freq = delta / 2 / np.pi
phase=np.pi/3
optimized_pulse_seq, qutip_result = qutip_optimize_wrapper(delta, omegad0, evo_time, n_ts, final_evo=True, target='sigmax', p_type='RND')

Final evolution
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 7.61519827e-06+1.46055723e-06j -9.90741776e-01+1.35759836e-01j]
 [-9.90743604e-01+1.35746496e-01j -6.94163893e-06+3.45515861e-06j]]



In [80]:
omegad0
freq_est
# wq0/16

4974450442.586346

In [98]:
getattr(armonk_backend.configuration(), 'hamiltonian')['vars']['omegad0'] = omegad0
getattr(armonk_backend.configuration(), 'hamiltonian')['vars']['wq0'] = wq0

delta_freq = delta / 2 / np.pi
armonk_backend.configuration().qubit_lo_freq=[freq_est - delta_freq]
# armonk_backend.configuration().qubit_lo_freq=[freq_est - delta_freq]

In [101]:
qutip_result.evo_full_final

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 7.61519827e-06+1.46055723e-06j -9.90741776e-01+1.35759836e-01j]
 [-9.90743604e-01+1.35746496e-01j -6.94163893e-06+3.45515861e-06j]]

In [117]:
gate, inst_map = pulse_to_gate_1q(backend, pulse.SamplePulse(optimized_pulse_seq))
pulse_aer_backend = qiskit.providers.aer.PulseSimulator(
    configuration=armonk_backend.configuration())
armonk_model = PulseSystemModel.from_backend(armonk_backend)

/Users/benrosand/anaconda3/envs/qutip/lib/python3.8/site-packages/qiskit/pulse/pulse_lib/sample_pulse.py:149: DeprecationWarning: Calling `SamplePulse` with a channel is deprecated. Instantiate the new `Play` instruction directly with a pulse and a channel. In this case, please use: `Play(SamplePulse(samples), DriveChannel(0))`.
  warnings.warn("Calling `{}` with a channel is deprecated. Instantiate the new `Play` "
/Users/benrosand/anaconda3/envs/qutip/lib/python3.8/site-packages/qiskit/pulse/pulse_lib/pulse.py:46: DeprecationWarning: Calling `SamplePulse` with a channel is deprecated. Instantiate the new `Play` instruction directly with a pulse and a channel. In this case, please use: `Play(SamplePulse(array([-0.99999074+0.j, -0.99933906+0.j,  0.92067025+0.j, ...,
        0.90854336+0.j, -0.99934313+0.j, -0.99999079+0.j])), DriveChannel(0))`.
  warnings.warn("Calling `{}` with a channel is deprecated. Instantiate the new `Play` "
/Users/benrosand/anaconda3/envs/qutip/lib/python3.8/si

In [ ]:
# q = QuantumRegister(1)
# c = ClassicalRegister(1)
# circ = qiskit.QuantumCircuit(q, c)
# circ.x(q)
# # circ.measure([0], [0])
# qpt_circs = process_tomography_circuits(circ, q)
# pulse_transp_circ = qiskit.transpile(qpt_circs, backend=pulse_aer_backend, basis_gates=armonk_backend.configuration().basis_gates + ['opt_gate'])
# gate_sched = qiskit.schedule(pulse_transp_circ, inst_map=inst_map, backend=pulse_aer_backend, meas_map=[[0]])

# default_jobs = [qiskit.execute(sched, pulse_aer_backend, shots=4096, system_model=armonk_model) for sched in gate_sched]

In [ ]:
# q = QuantumRegister(1)
# c = ClassicalRegister(1)
# circ = qiskit.QuantumCircuit(q, c)
# # circ.append(gate,q)
# circ.x(0)
# # circ.measure(q, c)
# qpt_circs = process_tomography_circuits(circ, q)
# pulse_transp_circ = qiskit.transpile(circ, backend=pulse_aer_backend, basis_gates=armonk_backend.configuration().basis_gates + ['opt_gate'])
# gate_sched = qiskit.schedule(pulse_transp_circ, inst_map=inst_map, backend=pulse_aer_backend, meas_map=[[0]])
# # default_jobs[0].result().get_counts()
# job = qiskit.execute(gate_sched, pulse_aer_backend, shots=4096, system_model=armonk_model)

In [ ]:
# job.result().get_counts()
# qpt_circs[6].draw()

In [ ]:
#NOTE we removed measurement for qpt?
q = QuantumRegister(1)
c = ClassicalRegister(1)
my_circ = qiskit.QuantumCircuit(q, c)
my_circ.append(gate,q)
# my_circ.x(q)
my_circ.measure([0], [0])
# my_circ.Acquire()
# acquire = pulse.Acquire(100, channel=pulse.AcquireChannel(
#         0), mem_slot=pulse.MemorySlot(0), name='acquire')


my_qpt_circs = process_tomography_circuits(my_circ, q)
my_transp_circ = qiskit.transpile(my_qpt_circs, backend=pulse_aer_backend, basis_gates=armonk_backend.configuration().basis_gates + ['opt_gate'])
my_gate_sched = qiskit.schedule(my_transp_circ, inst_map=inst_map, backend=pulse_aer_backend, meas_map=[[0]])

# my_jobs = [qiskit.execute(sched, pulse_aer_backend, shots=4096, system_model=armonk_model) for sched in my_gate_sched]
# my_jobs[0].result().get_counts()

In [ ]:
small_transp = qiskit.transpile(my_circ, backend=pulse_aer_backend, basis_gates=armonk_backend.configuration().basis_gates + ['opt_gate'])
small_gate_sched = qiskit.schedule(small_transp, inst_map=inst_map, backend=pulse_aer_backend, meas_map=[[0]])
# small_gate_sched += acquire << small_gate_sched.duration()
temp_job = qiskit.execute(small_gate_sched, pulse_aer_backend, shots=4096, system_model=armonk_model)

In [ ]:
temp_job.result().get_counts()

In [ ]:
# my_qpt_circs[1].draw()
# qpt_circs[1].draw()
pulse_transp_circ[4].draw()

In [ ]:
qpt_circs[4].draw()

In [ ]:
my_transp_circ[4].draw()

In [ ]:
default_tomo = ProcessTomographyFitter(default_jobs[0].result(), [pulse_transp_circ[0]])
my_tomo = ProcessTomographyFitter(my_jobs[0].result(), [my_transp_circ[0]])

for i,_ in enumerate(my_jobs[:-1]):
    default_tomo.add_data(default_jobs[i+1].result(), [pulse_transp_circ[i+1]])
    my_tomo.add_data(my_jobs[i+1].result(), [my_transp_circ[i+1]])

In [ ]:
q = QuantumRegister(1)
circ = qiskit.QuantumCircuit(q)
# circ.x(q[0])
circ.x(0)

# Run circuit on unitary simulator to find ideal unitary
unitary_job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = unitary_job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q)
ideal_job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=4000)

# ideal_transp_circ = qiskit.transpile(qpt_circs, backend=pulse_aer_backend, basis_gates=armonk_backend.configuration().basis_gates + ['opt_gate'])
# ideal_gate_sched = qiskit.schedule(ideal_transp_circ, inst_map=inst_map, backend=pulse_aer_backend, meas_map=[[0]])
# ideal_armonk_job = qiskit.execute(qpt_circs, armonk_backend, shots=4096)

# Extract tomography data so that counts are indexed by measurement configuration
ideal_qpt_tomo = ProcessTomographyFitter(ideal_job.result(), qpt_circs)

x_choi_lstsq = ideal_qpt_tomo.fit(method='lstsq')

In [ ]:
# ideal_qpt_tomo.data

In [ ]:
t = time.time()
my_choi_lstsq = my_tomo.fit(method='lstsq')
default_choi_lstsq = default_tomo.fit(method='lstsq')
ideal_choi_lstsq = ideal_qpt_tomo.fit(method='lstsq')

print('Least-Sq Fitter')
print('my fit fidelity (state):', state_fidelity(choi_ideal / 2, my_choi_lstsq.data / 2))
print('default fit fidelity (state):', state_fidelity(choi_ideal / 2, default_choi_lstsq.data / 2))
print('ideal fit fidelity (state):', state_fidelity(choi_ideal / 2, ideal_choi_lstsq.data / 2))

print('my fit time:', time.time() - t)
print('default fit fidelity (process):', np.real(process_fidelity(choi_ideal, default_choi_lstsq.data, require_cptp=False)))
print('my fit fidelity (process):', np.real(process_fidelity(choi_ideal, my_choi_lstsq.data, require_cptp=False)))

In [ ]:
default_tomo.data

In [ ]:
default_tomo.data

In [ ]:
my_tomo.data

In [ ]:
ideal_qpt_tomo.data